In [21]:
# Import the python modules
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from langchain_core.messages import HumanMessage, SystemMessage
from PIL import Image, ImageOps

load_dotenv()

True

In [22]:
# Open the image file
file = Image.open("images/lays_cropped.jpg")
image1 = ImageOps.grayscale(file)
image1.show()


file2 = Image.open("images/lays_magic_cropped.jpg")
image2 = ImageOps.grayscale(file2)
image2.show()

In [23]:
# Open the file in read mode
with open("src/prompts/label_prompt.txt", "r") as f:
    # Read the contents of the file
    file_contents = f.read()


print(file_contents)

label_extraction_prompt = file_contents

Parse the nutritional information and the list of ingredients of the product shown in the image. 

Use the nutritional information/nutritional fact label to identify the quantity of the following items:
Calories/Energy, Total Fat, Saturated Fat, Trans Fat, Cholesterol, Sodium, Total Carbohydrates, Dietary Fiber, Sugars, Protein, Vitamin A, Vitamin C, Calcium, and Iron.        
If the item is not present in the image, please mention that the item is not present in the image.

The output should contain only the nutritional information and the list of ingredients that is present on the product shown in the image. 
The output should be in a JSON format populated ONLY with the information that is available in the image. Make sure the JSON is valid and
do not add any additional information that is not present in the image.

Example of the output format:
{
    Nutritional information: {},
    Ingredients: ['items1', 'item2', 'item3']
}



In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-pro-vision")

# example
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": label_extraction_prompt,
        },
        {"type": "image_url", "image_url": image2},
    ]
)

response = llm.invoke([message])

In [7]:
generated_text = response.content
print(generated_text)

 ```JSON
{
 "Nutritional information": {
  "Energy": "547 kCal",
  "Total Fat": "35.7 g",
  "Saturated Fat": "14.1 g",
  "Trans Fat": "0.1 g",
  "Cholesterol": "0 mg",
  "Sodium": "942 mg",
  "Total Carbohydrates": "47.6 g",
  "Dietary Fiber": "Not Present",
  "Sugars": "Not Present",
  "Protein": "8.9 g",
  "Vitamin A": "Not Present",
  "Vitamin C": "Not Present",
  "Calcium": "Not Present",
  "Iron": "Not Present"
 },
 "Ingredients": [
  "Potato",
  "Edible Vegetable Oil",
  "Palm Oil",
  "Rice Bran Oil",
  "Spices & Condiments",
  "Onion Powder",
  "Chilli Powder",
  "Dried Mango Powder",
  "Coriander Seed Powder",
  "Ginger Powder",
  "Garlic Powder",
  "Black Pepper Powder",
  "Cumin Powder",
  "Turmeric Powder",
  "Iodised Salt",
  "Black Salt",
  "Sugar",
  "Tomato Puree",
  "Citric Acid",
  "Natural & Nature Identical Flavouring Substances"
 ]
}
```


In [24]:
#Load the models
llm = ChatGoogleGenerativeAI(model="gemini-pro")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [25]:
#Load the PDF and create chunks
loader = PyPDFLoader("docs/Appendix-E3-1-Table-A4.pdf")
text_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=250,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
pages = loader.load_and_split(text_splitter)

Created a chunk of size 819, which is longer than the specified 250
Created a chunk of size 261, which is longer than the specified 250
Created a chunk of size 500, which is longer than the specified 250
Created a chunk of size 342, which is longer than the specified 250


In [27]:
#Turn the chunks into embeddings and store them in Chroma
vectordb=Chroma.from_documents(pages, embeddings)

In [34]:
#Configure Chroma as a retriever with top_k=5
retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [35]:
#Create the retrieval chain
template = """
You are a helpful AI assistant.
Answer based on the context provided. 
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

#Invoke the retrieval chain
response=retrieval_chain.invoke({"input":"How amount of protien is recommended for a male between the age group 51+?"})

#Print the answer to the question
print(response["answer"])

56g
